In [2]:
#conda install -c conda-forge gdal

In [3]:
#Clear trash everytime before simulating new scene
#rm -rf $HOME/.local/share/Trash/files

In [1]:
import rasterio as rio
from rasterio.windows import Window
from matplotlib import pyplot as plt
import numpy as np
from itertools import product
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import os
from osgeo import gdal
from google.cloud import storage

In [2]:
#read standard spectra
birch = pd.read_csv("gs://hysim/CSV_spectra_srf/birch.csv")
black_spruce = pd.read_csv("gs://hysim/CSV_spectra_srf/black_spruce.csv")
trail = pd.read_csv("gs://hysim/CSV_spectra_srf/trail.csv")

In [3]:
birch = birch["Mean_birch_Reflectance"]
black_spruce = black_spruce["Mean_black_spruce_Reflectance"]
trail = trail["Mean_trail_Reflectance"]

print (trail.shape)

(2150,)


In [4]:
srf_mss = pd.read_csv("gs://hysim/CSV_spectra_srf/srf_mss.csv")

srf_mss=srf_mss.drop(srf_mss.columns[0:2], axis=1)
srf_mss_transpose=srf_mss.T
print (srf_mss.shape)
print (srf_mss_transpose.shape)
#print (srf_mss)

(2150, 10)
(10, 2150)


In [5]:
#resample standard spectra

birch = birch.values.reshape(2150,1)
black_spruce = black_spruce.values.reshape(2150,1)
trail = trail.values.reshape(2150,1)


#matrix multiplication 
R_birch=np.dot(srf_mss_transpose, birch)
R_black_spruce=np.dot(srf_mss_transpose, black_spruce)
R_trail=np.dot(srf_mss_transpose, trail)


RM = np.concatenate((R_birch, R_black_spruce, R_trail), axis=1)

print (RM.shape)

print (RM)

RM_t=RM.T
#==================================
#part second hyperspectral simulation
srf_hss = pd.read_csv("gs://hysim/CSV_spectra_srf/AVIRIS_SRF.csv",sep=',')
srf_hss=srf_hss.drop(srf_hss.columns[0:2], axis=1)
srf_hss_transpose=srf_hss.T
print (srf_hss.shape)
print (srf_hss_transpose.shape)

#matrix multiplication 

Rh_birch=np.dot(srf_hss_transpose, birch)
Rh_black_spruce=np.dot(srf_hss_transpose, black_spruce)
Rh_trail=np.dot(srf_hss_transpose, trail)

RH = np.concatenate((Rh_birch, Rh_black_spruce, Rh_trail), axis=1)

print (RM_t.shape)

(10, 3)
[[ 271.43572918  276.05403922  641.73142581]
 [ 416.66909055  284.71903591  460.58216114]
 [ 137.43311212  157.08899757  485.38986254]
 [ 227.29035927  159.49784909  241.32303001]
 [ 833.09336933  397.08715302  241.40177744]
 [1376.36133012  640.58713463  312.01643138]
 [6806.52191456 3258.29623104 1474.76493376]
 [1667.96711533  812.32917947  350.98433605]
 [4035.30546788 1473.23975678 1834.32430529]
 [3759.3070371  1265.7171196  2823.0644978 ]]
(2150, 425)
(425, 2150)
(3, 10)


In [6]:
#!gsutil cp gs://testbucket_111221/S2BL2_T06WWS.tif /home/jupyter/input/ 

In [7]:
#change path: Path for Sentinel image
src_path = Path("/home/jupyter/input/S2AL2_T06WVS.tif")  # Specify input image path"
#===========================

dst_tag = "HS"
dst_dir = Path("/home/jupyter/output")
dst_dir.mkdir(mode=0o755, parents=True, exist_ok=True)
tiles_dir = dst_dir / "Tiles"
tiles_dir.mkdir(mode=0o755, parents=True, exist_ok=True)
win_height = 2048  # Change as required 2048
win_width = 2048  # Change as required

In [8]:
with rio.open(src_path, 'r') as src:
    meta = src.profile.copy()
    img_h = src.height
    img_w = src.width
    big_win = Window(row_off=0, col_off=0, height=img_h, width=img_w)
    r_offsets = list(range(0, src.height, win_height))
    c_offsets = list(range(0, src.width, win_width))
    r_indexes = list(range(len(r_offsets)))
    c_indexes = list(range(len(c_offsets)))
    offsets = list(product(r_offsets, c_offsets))
    indexes = list(product(r_indexes, c_indexes))
    pointers = list(zip(indexes, offsets))
    
    # update meta for output image as required
    meta['count'] = RH.shape[0]
    meta['dtype'] = np.float32
    meta['nodata'] = np.nan
    meta['BIGTIFF'] = True
    # meta['compress'] = 'zstd'
    # meta['predictor'] = 3
    tiles = list()

    for (i, j), (r_off, c_off) in tqdm(pointers):
      win = Window(
        row_off=r_off, col_off=c_off, height=win_height, width=win_width
      ).intersection(big_win)
      img = src.read(window=win, boundless=False, masked=True) # This is a 3D array (band x Rowsx Cols)
      
      # processing 
      dp = img.filled()
      dp = np.moveaxis(dp, -1, 0)
      dp = np.dot(RM_t, dp)
      dp = np.moveaxis(dp, -1, 0)
      dp = np.dot(np.linalg.inv(np.dot(RM_t,RM)), dp)
      dp = np.moveaxis(dp, -1, 0)
      dp = np.dot(RH, dp)
      dp = np.moveaxis(dp, -1, 1)

      out = dp.astype(meta['dtype'])
      mask = np.any(img.mask, axis=0, keepdims=True)
      mask = np.repeat(mask, meta['count'], axis=0)
      out[mask] = meta['nodata']

      # Write processed array to file
      dst_path = tiles_dir / "{}_{}_{}_{}{}".format(
        src_path.stem, dst_tag, i, j, src_path.suffix
      )
      meta['height'] = win.height
      meta['width'] = win.width
      meta['transform'] = src.window_transform(win)
      with rio.open(dst_path, 'w', **meta) as dst:
        dst.write(out) # Band id needs to be soecified in case of writing a 2D array 
      tiles.append(str(dst_path.relative_to(dst_dir)))
    os.chdir(dst_dir)
    vrt_path = dst_dir / '{}_{}.{}'.format(src_path.stem, dst_tag, "VRT")
    vrt_options = gdal.BuildVRTOptions(resampleAlg='near', addAlpha=False)
    ds = gdal.BuildVRT(
    str(vrt_path.relative_to(dst_dir)), tiles, options=vrt_options
    )
    ds.FlushCache()

100%|██████████| 36/36 [2:04:05<00:00, 206.81s/it]  
